In [3]:
import requests 
import logging 
from typing import List, Dict 

In [4]:
logging.basicConfig(level=logging.INFO) 
logger = logging.getLogger(__name__)

In [5]:
class MOEXService:
    # Официальный бесплатный API Мосбиржи
    BASE_URL = "https://iss.moex.com/iss"

    @staticmethod
    def get_top_stocks(limit: int = 50) -> List[Dict]:
        """
        Получает список топ-акций с основного режима торгов (TQBR).
        Нам нужны: Тикер (SECID), Название (SHORTNAME) и Цена (PREVPRICE).
        """
        try:
            # TQBR - это основной режим торгов акциями и паями
            url = f"{MOEXService.BASE_URL}/engines/stock/markets/shares/boards/TQBR/securities.json"
            
            # Делаем запрос
            params = {'limit': limit} # Берем топ-50, чтобы не грузить лишнее
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()

            data = response.json()
            
            # Структура ответа MOEX специфична (список списков), парсим её:
            columns = data['securities']['columns']
            rows = data['securities']['data']
            
            # Находим индексы нужных колонок
            secid_idx = columns.index('SECID')
            name_idx = columns.index('SHORTNAME')
            price_idx = columns.index('PREVPRICE') # Цена закрытия вчерашнего дня
            
            stocks = []
            for row in rows:
                # Собираем чистый словарь
                stock = {
                    'ticker': row[secid_idx],
                    'name': row[name_idx],
                    'price': row[price_idx] if row[price_idx] else 0.0
                }
                stocks.append(stock)
            
            logger.info(f"✅ Успешно загружено {len(stocks)} акций с MOEX")
            return stocks

        except Exception as e:
            logger.error(f"❌ Ошибка при получении данных MOEX: {e}")
            return []


In [6]:
some = MOEXService()

In [7]:
some.get_top_stocks()

INFO:__main__:✅ Успешно загружено 259 акций с MOEX


[{'ticker': 'ABIO', 'name': 'iАРТГЕН ао', 'price': 70},
 {'ticker': 'ABRD', 'name': 'АбрауДюрсо', 'price': 157.6},
 {'ticker': 'AFKS', 'name': 'Система ао', 'price': 13.024},
 {'ticker': 'AFLT', 'name': 'Аэрофлот', 'price': 55.57},
 {'ticker': 'AKRN', 'name': 'Акрон', 'price': 17854},
 {'ticker': 'ALRS', 'name': 'АЛРОСА ао', 'price': 38.99},
 {'ticker': 'AMEZ', 'name': 'АшинскийМЗ', 'price': 68.05},
 {'ticker': 'APRI', 'name': 'АПРИ', 'price': 12.852},
 {'ticker': 'APTK', 'name': 'Аптеки36и6', 'price': 8.3},
 {'ticker': 'AQUA', 'name': 'ИНАРКТИКА', 'price': 473},
 {'ticker': 'ARSA', 'name': 'Арсагера', 'price': 9},
 {'ticker': 'ASSB', 'name': 'АстрЭнСб', 'price': 2.58},
 {'ticker': 'ASTR', 'name': 'iАстра ао', 'price': 281.6},
 {'ticker': 'AVAN', 'name': 'Авангрд-ао', 'price': 676},
 {'ticker': 'BANE', 'name': 'Башнефт ао', 'price': 1470},
 {'ticker': 'BANEP', 'name': 'Башнефт ап', 'price': 911.5},
 {'ticker': 'BELU', 'name': 'НоваБев ао', 'price': 420.4},
 {'ticker': 'BISVP', 'name': 